In [23]:
import geopandas as gpd
from shapely.wkt import loads
import h3
from shapely.geometry import box, Polygon


with open("../data/trento_city_boundary.wkt", "r") as file:
    city_boundary = loads(file.read())

geo_file_path = '../data/kontur_population_IT_20220630.gpkg'

pop = gpd.read_file(geo_file_path)
pop = pop.to_crs("EPSG:4326")
pop_gdf = pop[pop.geometry.intersects(city_boundary)]
print(pop_gdf)


# Function to create a hexagonal grid within the city boundaries
def create_hex_grid_within_city_bounds(city_boundary, resolution):
    hexagons = h3.polyfill(city_boundary.__geo_interface__, resolution, geo_json_conformant=True)

    filtered_hexagons = []
    for h in hexagons:
        hex_polygon = Polygon(h3.h3_to_geo_boundary(h, geo_json=True))
        if hex_polygon.intersects(city_boundary):
            filtered_hexagons.append(hex_polygon)

    hex_grid = gpd.GeoDataFrame([{'geometry': hexagon} for hexagon in filtered_hexagons])
    hex_grid.crs = 'EPSG:4326'

    return hex_grid

hex_grid_within_city = create_hex_grid_within_city_bounds(city_boundary, 9)
hex_grid_projected = hex_grid_within_city.to_crs('EPSG: 32632')
print(hex_grid_projected)

                     h3  population  \
74013   881ea481d3fffff         1.0   
74021   881ea480cdfffff      1648.0   
74022   881ea480b1fffff       919.0   
74032   881ea4856bfffff      2253.0   
74033   881ea48565fffff       546.0   
...                 ...         ...   
123903  881ea48465fffff        24.0   
123923  881ea4842dfffff       164.0   
133484  881ea495b3fffff         1.0   
133487  881ea495a1fffff       365.0   
133499  881ea49581fffff         7.0   

                                                 geometry  
74013   POLYGON ((11.04685 46.03190, 11.04498 46.02729...  
74021   POLYGON ((11.05202 46.07516, 11.05015 46.07055...  
74022   POLYGON ((11.09916 46.03879, 11.09729 46.03418...  
74032   POLYGON ((11.13982 46.04741, 11.13794 46.04280...  
74033   POLYGON ((11.12251 46.03531, 11.12064 46.03070...  
...                                                   ...  
123903  POLYGON ((11.18311 46.07764, 11.18123 46.07303...  
123923  POLYGON ((11.19781 46.06810, 11.19593 46.06

In [24]:
pop_gdf = pop_gdf.to_crs(hex_grid_projected.crs)

# Spatial join
joined_gdf = gpd.sjoin(hex_grid_projected, pop_gdf, how="left", predicate='intersects')

# Since one hexagon might intersect with multiple population polygons,
# we aggregate the population data for each hexagon.
# Here we sum the populations, but you might choose a different method
# depending on your specific requirements.
hex_grid_with_population = joined_gdf.groupby(joined_gdf.index).agg({'population': 'sum'})
hex_grid_projected = hex_grid_projected.merge(hex_grid_with_population, left_index=True, right_index=True)
hex_grid_projected['population'] = hex_grid_projected['population'].fillna(0)
print(hex_grid_projected.head())


                                            geometry  population
0  POLYGON ((665242.309 5110752.670, 665260.260 5...       448.0
1  POLYGON ((657212.867 5104833.944, 657230.733 5...       483.0
2  POLYGON ((662888.431 5099370.773, 662906.358 5...        38.0
3  POLYGON ((659028.771 5104495.406, 659046.656 5...      2354.0
4  POLYGON ((667801.307 5104250.934, 667819.285 5...       993.0
